![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **Getting Started Code For [Data Sprint #48](https://dphi.tech/challenges/data-sprint-48-car-body-type-classifier/162/overview/about) on DPhi**

## Download the images
We are given google drive link in the '[Data](https://dphi.tech/challenges/data-sprint-48-car-body-type-classifier/162/data)' section of problem page which has all the required train images (to build the model) and test images to predict the label of these images and submit the predictions on the [DPhi platform](https://dphi.tech/challenges/data-sprint-48-car-body-type-classifier/162/overview/about).


In [ ]:
# Loads the data from AWS to Colab
!wget https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/standford_cars.zip

# Unzip our data 
!unzip standford_cars.zip

Streaming output truncated to the last 5000 lines.
  inflating: standford_cars/train/Image_163.jpg  
  inflating: standford_cars/train/Image_1630.jpg  
  inflating: standford_cars/train/Image_1631.jpg  
  inflating: standford_cars/train/Image_1632.jpg  
  inflating: standford_cars/train/Image_1633.jpg  
  inflating: standford_cars/train/Image_1634.jpg  
  inflating: standford_cars/train/Image_1635.jpg  
  inflating: standford_cars/train/Image_1636.jpg  
  inflating: standford_cars/train/Image_1637.jpg  
  inflating: standford_cars/train/Image_1638.jpg  
  inflating: standford_cars/train/Image_1639.jpg  
  inflating: standford_cars/train/Image_164.jpg  
  inflating: standford_cars/train/Image_1640.jpg  
  inflating: standford_cars/train/Image_1641.jpg  
  inflating: standford_cars/train/Image_1642.jpg  
  inflating: standford_cars/train/Image_1643.jpg  
  inflating: standford_cars/train/Image_1644.jpg  
  inflating: standford_cars/train/Image_1645.jpg  
  inflating: standford_cars/train

We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [ ]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'. The labels of train images are given in a csv file 'Train.csv' with respective image id (i.e. image file name).

#### Getting the labels of the images

In [ ]:
labels = pd.read_csv("/content/standford_cars/Training_set.csv")   # loading the labels
labels.head()           # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,Sedan
1,Image_2.jpg,Sedan
2,Image_3.jpg,Convertible
3,Image_4.jpg,Convertible
4,Image_5.jpg,Sedan


In [ ]:
labels.tail()            # will display the last five rows in labels dataframe

,filename,label
5695,Image_5696.jpg,Convertible
5696,Image_5697.jpg,Convertible
5697,Image_5698.jpg,Other
5698,Image_5699.jpg,Cab
5699,Image_5700.jpg,Sedan


#### Getting images file path

In [ ]:
file_paths = [[fname, '/content/standford_cars/train/' + fname] for fname in labels['filename']]

#### Confirming if no. of labels is equal to no. of images

In [ ]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  5700 matches the number of filenames i.e.  5700


#### Converting the file_paths to dataframe

In [ ]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/standford_cars/train/Image_1.jpg
1,Image_2.jpg,/content/standford_cars/train/Image_2.jpg
2,Image_3.jpg,/content/standford_cars/train/Image_3.jpg
3,Image_4.jpg,/content/standford_cars/train/Image_4.jpg
4,Image_5.jpg,/content/standford_cars/train/Image_5.jpg


#### Combining the labels with the images

In [ ]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()       

,filename,filepaths,label
0,Image_1.jpg,/content/standford_cars/train/Image_1.jpg,Sedan
1,Image_2.jpg,/content/standford_cars/train/Image_2.jpg,Sedan
2,Image_3.jpg,/content/standford_cars/train/Image_3.jpg,Convertible
3,Image_4.jpg,/content/standford_cars/train/Image_4.jpg,Convertible
4,Image_5.jpg,/content/standford_cars/train/Image_5.jpg,Sedan


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

The 'train_data' dataframe contains all the image id, their locations and their respective labels. Now the training data is ready.

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [ ]:
data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

In [ ]:
# image pixels of a image
data[5]

[array([[169, 179, 181, ..., 214, 214, 209],
        [168, 178, 179, ..., 219, 219, 221],
        [174, 183, 185, ..., 226, 226, 219],
        ...,
        [ 66,  91,  80, ...,  89,  89,  91],
        [ 71,  79,  83, ...,  96,  94,  96],
        [ 81,  73,  65, ...,  89,  84,  99]], dtype=uint8), 1]

#### Shuffle the data

In [ ]:
np.random.shuffle(data)

#### Separating the images and labels


In [ ]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 503,  725,  761,  388,  175,  426, 1006, 1335,  177,  204]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [ ]:
x =  x.reshape(-1, 100, 100, 1)

In [ ]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [ ]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(131, activation='sigmoid')
])

In [ ]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs=2, batch_size=1)

Epoch 1/250
200/200 [==============================] - 7s 31ms/step - loss: 2.1870 - accuracy: 0.2165 - val_loss: 2.1166 - val_accuracy: 0.2310
Epoch 2/250
200/200 [==============================] - 6s 30ms/step - loss: 2.1219 - accuracy: 0.2268 - val_loss: 2.1051 - val_accuracy: 0.2427
Epoch 3/250
200/200 [==============================] - 6s 30ms/step - loss: 2.1103 - accuracy: 0.2312 - val_loss: 2.0990 - val_accuracy: 0.2281
Epoch 4/250
200/200 [==============================] - 6s 30ms/step - loss: 2.0988 - accuracy: 0.2312 - val_loss: 2.0956 - val_accuracy: 0.2544
Epoch 5/250
200/200 [==============================] - 6s 30ms/step - loss: 2.0931 - accuracy: 0.2397 - val_loss: 2.0872 - val_accuracy: 0.2164
Epoch 6/250
200/200 [==============================] - 6s 30ms/step - loss: 2.0925 - accuracy: 0.2350 - val_loss: 2.1090 - val_accuracy: 0.2427
Epoch 7/250
200/200 [==============================] - 6s 30ms/step - loss: 2.0848 - accuracy: 0.2293 - val_loss: 2.1330 - val_accuracy:

## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [ ]:
cnn.evaluate(X_val, y_val)

54/54 [==============================] - 6s 101ms/step - loss: 2.1760 - accuracy: 0.2374


[2.175957202911377, 0.23742690682411194]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [ ]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/standford_cars/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


#### Getting images file path

In [ ]:
file_paths = [[fname, '/content/standford_cars/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [ ]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  2444 matches the number of file paths i.e.  2444


#### Converting the file_paths to dataframe

In [ ]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/standford_cars/test/Image_1.jpg
1,Image_2.jpg,/content/standford_cars/test/Image_2.jpg
2,Image_3.jpg,/content/standford_cars/test/Image_3.jpg
3,Image_4.jpg,/content/standford_cars/test/Image_4.jpg
4,Image_5.jpg,/content/standford_cars/test/Image_5.jpg


## Data Pre-processing on test_data


In [ ]:
test_pixel_data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):
  
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array

  test_pixel_data.append(new_img_array)

In [ ]:
test_pixel_data = np.array(test_pixel_data)

In [ ]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
pred = cnn.predict(test_pixel_data)

In [ ]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [ ]:
predictions = le.inverse_transform(prediction)

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/data-sprint-48-car-body-type-classifier/162/overview/about)** to make the submission.